In [1]:
%load_ext autoreload
%autoreload 2
from indices import MainDataset
from neighbors import Neighborhoods
import functions
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from catboost import CatBoostRegressor
from catboost import Pool
from metric import metrics_stat

In [2]:
train_dataset = MainDataset("dataset/train.csv")
test_dataset = MainDataset("dataset/test.csv", need_index = False)

/Users/ivan.bragin/projects/raif/venv/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3263: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [3]:
neighborhoods = Neighborhoods(train_dataset.index)
neighborhoods1 = Neighborhoods(train_dataset.index1)

In [4]:
# for i, o in enumerate(train_dataset.all_objects[:10000]):
#     if i % 1000 == 0:
#         print(i)
#     neighbor = neighborhoods.get_haversine_closest(o, 7)
#     for f_name, f in functions.items():
#         prices = [f(o, x[0], x[1]) for x in neighbor]
#         o.neighbors_scores[f_name] = np.mean(prices)

In [5]:
for prefix, neighborhoods_item in [("", neighborhoods), ("P1_", neighborhoods1)]:
    for i, o in enumerate(train_dataset.all_objects):
#         o.neighbors_scores = {}
        if o.row["price_type"] != 1:
            continue
        if i % 1000 == 0:
            print(i)
        neighbor = neighborhoods_item.get_haversine_closest(o, 7)
        for f_name, f in functions.functions.items():
            prices = [f(o, x[0], x[1]) for x in neighbor]
            prices = [x for x in prices if x > 0]
            if len(prices) > 0:
                price_median = np.median(prices)
                price_max = np.max(prices)
                price_min = np.min(prices)

            else:
                price_median = 0
                price_max = 0
                price_min = 0
            o.neighbors_scores[prefix + f_name+"_median"] = price_median
            o.neighbors_scores[prefix + f_name+"_max"] = price_max
            o.neighbors_scores[prefix + f_name+"_min"] = price_min
    

253000
254000
255000
256000
253000
254000
255000
256000


In [6]:
for prefix, neighborhoods_item in [("", neighborhoods), ("P1_", neighborhoods1)]:
    for i, o in enumerate(test_dataset.all_objects):
        if o.row["price_type"] != 1:
            continue
        if i % 1000 == 0:
            print(i)
        neighbor = neighborhoods_item.get_haversine_closest(o, 7)
        for f_name, f in functions.functions.items():
            prices = [f(o, x[0], x[1]) for x in neighbor]
            prices = [x for x in prices if x > 0]
            if len(prices) > 0:
                price_median = np.median(prices)
                price_max = np.max(prices)
                price_min = np.min(prices)

            else:
                price_median = 0
                price_max = 0
                price_min = 0
            o.neighbors_scores[prefix + f_name+"_median"] = price_median
            o.neighbors_scores[prefix + f_name+"_max"] = price_max
            o.neighbors_scores[prefix + f_name+"_min"] = price_min

0
1000
2000
0
1000
2000


In [7]:
raw_dense_features = [
#     'osm_historic_points_in_0.005',
#     'osm_culture_points_in_0.0075',
        'osm_amenity_points_in_0.001',
       'osm_amenity_points_in_0.005', 'osm_amenity_points_in_0.0075',
       'osm_amenity_points_in_0.01', 'osm_building_points_in_0.001',
       'osm_building_points_in_0.005', 'osm_building_points_in_0.0075',
       'osm_building_points_in_0.01', 'osm_catering_points_in_0.001',
       'osm_catering_points_in_0.005', 'osm_catering_points_in_0.0075',
       'osm_catering_points_in_0.01', 'osm_city_closest_dist',
        'osm_city_nearest_population',
       'osm_crossing_closest_dist', 'osm_crossing_points_in_0.001',
       'osm_crossing_points_in_0.005', 'osm_crossing_points_in_0.0075',
       'osm_crossing_points_in_0.01', 'osm_culture_points_in_0.001',
       'osm_culture_points_in_0.005', 
       'osm_culture_points_in_0.01', 'osm_finance_points_in_0.001',
       'osm_finance_points_in_0.005', 'osm_finance_points_in_0.0075',
       'osm_finance_points_in_0.01', 'osm_healthcare_points_in_0.005',
       'osm_healthcare_points_in_0.0075', 'osm_healthcare_points_in_0.01',
        'osm_historic_points_in_0.0075',
       'osm_historic_points_in_0.01', 'osm_hotels_points_in_0.005',
       'osm_hotels_points_in_0.0075', 'osm_hotels_points_in_0.01',
       'osm_leisure_points_in_0.005', 'osm_leisure_points_in_0.0075',
       'osm_leisure_points_in_0.01', 'osm_offices_points_in_0.001',
       'osm_offices_points_in_0.005', 'osm_offices_points_in_0.0075',
       'osm_offices_points_in_0.01', 'osm_shops_points_in_0.001',
       'osm_shops_points_in_0.005', 'osm_shops_points_in_0.0075',
       'osm_shops_points_in_0.01', 'osm_subway_closest_dist',
       'osm_train_stop_closest_dist', 'osm_train_stop_points_in_0.005',
       'osm_train_stop_points_in_0.0075', 'osm_train_stop_points_in_0.01',
       'osm_transport_stop_closest_dist', 'osm_transport_stop_points_in_0.005',
       'osm_transport_stop_points_in_0.0075',
       'osm_transport_stop_points_in_0.01',
                      'reform_count_of_houses_1000', 'reform_count_of_houses_500',
       'reform_house_population_1000', 'reform_house_population_500',
       'reform_mean_floor_count_1000', 'reform_mean_floor_count_500',
       'reform_mean_year_building_1000', 'reform_mean_year_building_500',
       'total_square', "lat", "lng"
       ]

raw_cat_features = ['city', 'osm_city_nearest_name', 'region', 'realty_type', 'street', 'floor']

bad_f = set(["F_square_False_min", "F_square_False_median"])
def getX(o):
    x = []
    for f_name, score in o.neighbors_scores.items():
        if f_name not in bad_f and "P1_kk" not in f_name:
            x.append(score)
    for n in raw_dense_features:
        x.append(o.row[n])
    for n in raw_cat_features:
        x.append(o.row[n])
    return pd.Series(x)


In [8]:
columns = []
for p in ["",
          "P1_"
         ]:
    for t in ["_median", "_min", "_max"]:
        columns.extend([p + f_name + t for f_name, f in functions.functions.items() ])
columns = [c for c in columns if c not in bad_f]
    # columns = []
columns.extend(raw_dense_features)
columns.extend(raw_cat_features)

In [9]:
# X0 = [getX(o) for o in train_dataset.all_objects[:1] if o.row["price_type"] == 0]
# X0 = pd.concat(X0, axis=1).T
# X0.columns=columns
# Y0 = [o.getY() for o in train_dataset.all_objects if o.row["price_type"] == 0]
# X0["floor"] = X0["floor"].astype("str")

In [10]:
X1 = [getX(o) for o in train_dataset.all_objects if o.row["price_type"] == 1]
X1 = pd.concat(X1, axis=1).T
X1.columns=columns
Y1 = [o.getY() for o in train_dataset.all_objects if o.row["price_type"] == 1]
X1 = X1.fillna(-999)


In [11]:
Xtest1 = [getX(o) for o in test_dataset.all_objects]
Xtest1 = pd.concat(Xtest1, axis=1).T
Xtest1.columns=columns
Xtest1 = Xtest1.fillna(-999)

In [49]:
# X0 = X0.fillna(-999)
X_train, X_val, Y_train, Y_val = train_test_split(X1, Y1, test_size=0.1, random_state=42)

In [13]:
import keras.backend as K

def custom_mse(y_true, y_pred):
 
    # calculating squared difference between target and predicted values 
    loss = K.square(y_pred - y_true)  # (batch_size, 2)
    
    # multiplying the values with weights along batch dimension
    loss = loss * [0.3, 0.7]          # (batch_size, 2)
                
    # summing both loss values along batch dimension 
    loss = K.sum(loss, axis=1)        # (batch_size,)
    
    return loss

ModuleNotFoundError: No module named 'keras'

In [1]:
pip install keras

Looking in indexes: https://pypi.org/simple, https://nexus.odkl.ru/repository/pypi-dev-hosted/simple
  Retrying (Retry(total=4, connect=None, read=None, redirect=None, status=None)) after connection broken by 'ConnectTimeoutError(<pip._vendor.urllib3.connection.VerifiedHTTPSConnection object at 0x7f83bc107278>, 'Connection to nexus.odkl.ru timed out. (connect timeout=15)')': /repository/pypi-dev-hosted/simple/keras/
  Retrying (Retry(total=3, connect=None, read=None, redirect=None, status=None)) after connection broken by 'ConnectTimeoutError(<pip._vendor.urllib3.connection.VerifiedHTTPSConnection object at 0x7f83bc107320>, 'Connection to nexus.odkl.ru timed out. (connect timeout=15)')': /repository/pypi-dev-hosted/simple/keras/
  Retrying (Retry(total=2, connect=None, read=None, redirect=None, status=None)) after connection broken by 'ConnectTimeoutError(<pip._vendor.urllib3.connection.VerifiedHTTPSConnection object at 0x7f83bc107400>, 'Connection to nexus.odkl.ru timed out. (connect 

In [ ]:
def get_input_output(m, s, event_len, mask=True):
    i = L.Input(event_len)
    o = L.Embedding(m+1, s, input_length=event_len, mask_zero=mask)(i)
    return i, o

def get_model():

    
    # 3. числовые фичи как в xgboost
    dense_input = L.Input(len(dense_features))
    x = L.Dense(128, activation="relu")(dense_input)
    
    # объединяем и классифицируем
    x = L.Concatenate()([x,svd,svd1,svd2,svd3,L.Flatten()(clientTypeOut),L.Flatten()(contentTypeOut)])
    x = L.Dense(128, activation="relu")(x)
    x = L.Dense(1, activation="sigmoid")(x)
    return Model([dense_input], x)
model = get_model()
model.summary()

In [84]:

train_data = Pool(data=X_train,
                  cat_features=raw_cat_features,
                  label=Y_train)

val_data = Pool(data=X_val,
                cat_features=raw_cat_features,
                  label=Y_val)

In [92]:
cat_model = CatBoostRegressor(
    learning_rate=0.025,
    iterations=7000
)
cat_model.fit(train_data,eval_set=val_data, use_best_model=False)



0:	learn: 88575.0311264	test: 83411.2484986	best: 83411.2484986 (0)	total: 27.3ms	remaining: 3m 11s
1:	learn: 87744.1089240	test: 82424.5500122	best: 82424.5500122 (1)	total: 46.8ms	remaining: 2m 43s
2:	learn: 86917.1920709	test: 81323.5317504	best: 81323.5317504 (2)	total: 66.1ms	remaining: 2m 34s
3:	learn: 86149.9356022	test: 80492.9018783	best: 80492.9018783 (3)	total: 95ms	remaining: 2m 46s
4:	learn: 85361.7601564	test: 79395.3635722	best: 79395.3635722 (4)	total: 110ms	remaining: 2m 34s
5:	learn: 84662.3551014	test: 78336.3927011	best: 78336.3927011 (5)	total: 121ms	remaining: 2m 21s
6:	learn: 84064.9583810	test: 77476.1902109	best: 77476.1902109 (6)	total: 147ms	remaining: 2m 26s
7:	learn: 83362.8809887	test: 76500.2246200	best: 76500.2246200 (7)	total: 164ms	remaining: 2m 23s
8:	learn: 82766.1975875	test: 75554.8779264	best: 75554.8779264 (8)	total: 180ms	remaining: 2m 19s
9:	learn: 82149.0585095	test: 74669.4232078	best: 74669.4232078 (9)	total: 202ms	remaining: 2m 20s
10:	lear

88:	learn: 61174.0772538	test: 41319.9756347	best: 41319.9756347 (88)	total: 1.24s	remaining: 1m 36s
89:	learn: 61050.8233387	test: 41064.6488660	best: 41064.6488660 (89)	total: 1.25s	remaining: 1m 36s
90:	learn: 60680.9951467	test: 40961.9852146	best: 40961.9852146 (90)	total: 1.27s	remaining: 1m 36s
91:	learn: 60492.2922704	test: 40794.7721851	best: 40794.7721851 (91)	total: 1.28s	remaining: 1m 36s
92:	learn: 60327.3664032	test: 40689.6525972	best: 40689.6525972 (92)	total: 1.29s	remaining: 1m 35s
93:	learn: 60217.5780358	test: 40476.9955937	best: 40476.9955937 (93)	total: 1.31s	remaining: 1m 36s
94:	learn: 59986.0986335	test: 40408.7107840	best: 40408.7107840 (94)	total: 1.32s	remaining: 1m 35s
95:	learn: 59887.7854726	test: 40222.0259261	best: 40222.0259261 (95)	total: 1.33s	remaining: 1m 35s
96:	learn: 59836.7702299	test: 40127.6743689	best: 40127.6743689 (96)	total: 1.34s	remaining: 1m 35s
97:	learn: 59798.2323469	test: 40046.2477936	best: 40046.2477936 (97)	total: 1.35s	remainin

181:	learn: 51735.3036719	test: 34524.8171739	best: 34524.8171739 (181)	total: 2.49s	remaining: 1m 33s
182:	learn: 51676.4471151	test: 34422.6075096	best: 34422.6075096 (182)	total: 2.5s	remaining: 1m 33s
183:	learn: 51622.4111129	test: 34318.6947131	best: 34318.6947131 (183)	total: 2.52s	remaining: 1m 33s
184:	learn: 51614.8548497	test: 34316.0299635	best: 34316.0299635 (184)	total: 2.53s	remaining: 1m 33s
185:	learn: 51575.8089608	test: 34280.1752131	best: 34280.1752131 (185)	total: 2.54s	remaining: 1m 33s
186:	learn: 51474.0826332	test: 34145.6342571	best: 34145.6342571 (186)	total: 2.56s	remaining: 1m 33s
187:	learn: 51436.3341817	test: 34125.4389295	best: 34125.4389295 (187)	total: 2.57s	remaining: 1m 33s
188:	learn: 51358.6233383	test: 34111.9353586	best: 34111.9353586 (188)	total: 2.58s	remaining: 1m 33s
189:	learn: 51283.4383554	test: 34097.6059920	best: 34097.6059920 (189)	total: 2.6s	remaining: 1m 33s
190:	learn: 51246.2476905	test: 34082.4058732	best: 34082.4058732 (190)	tot

273:	learn: 46545.5899677	test: 31281.3570410	best: 31279.9299390 (272)	total: 3.49s	remaining: 1m 25s
274:	learn: 46507.2371180	test: 31271.5085421	best: 31271.5085421 (274)	total: 3.51s	remaining: 1m 25s
275:	learn: 46472.4293514	test: 31258.2269483	best: 31258.2269483 (275)	total: 3.52s	remaining: 1m 25s
276:	learn: 46424.7328666	test: 31256.7212156	best: 31256.7212156 (276)	total: 3.53s	remaining: 1m 25s
277:	learn: 46374.9665423	test: 31160.9814869	best: 31160.9814869 (277)	total: 3.54s	remaining: 1m 25s
278:	learn: 46362.2281483	test: 31041.8789965	best: 31041.8789965 (278)	total: 3.55s	remaining: 1m 25s
279:	learn: 46332.3033864	test: 31034.7496478	best: 31034.7496478 (279)	total: 3.56s	remaining: 1m 25s
280:	learn: 46302.1882013	test: 31007.3590822	best: 31007.3590822 (280)	total: 3.58s	remaining: 1m 25s
281:	learn: 46265.9988336	test: 30949.5295034	best: 30949.5295034 (281)	total: 3.58s	remaining: 1m 25s
282:	learn: 46253.7934769	test: 30900.2549078	best: 30900.2549078 (282)	t

355:	learn: 42564.9654493	test: 28758.0927205	best: 28758.0927205 (355)	total: 4.49s	remaining: 1m 23s
356:	learn: 42548.5097678	test: 28757.7543169	best: 28757.7543169 (356)	total: 4.5s	remaining: 1m 23s
357:	learn: 42512.4713500	test: 28753.5281725	best: 28753.5281725 (357)	total: 4.53s	remaining: 1m 24s
358:	learn: 42494.9969127	test: 28728.4543601	best: 28728.4543601 (358)	total: 4.54s	remaining: 1m 23s
359:	learn: 42280.5030234	test: 28701.3120487	best: 28701.3120487 (359)	total: 4.55s	remaining: 1m 23s
360:	learn: 42224.2834240	test: 28677.3055035	best: 28677.3055035 (360)	total: 4.57s	remaining: 1m 23s
361:	learn: 42145.3363330	test: 28657.1348953	best: 28657.1348953 (361)	total: 4.58s	remaining: 1m 23s
362:	learn: 42131.0893257	test: 28652.3401142	best: 28652.3401142 (362)	total: 4.59s	remaining: 1m 23s
363:	learn: 42073.9803076	test: 28649.7114871	best: 28649.7114871 (363)	total: 4.61s	remaining: 1m 24s
364:	learn: 42029.2363881	test: 28654.1880459	best: 28649.7114871 (363)	to

...

6622:	learn: 4201.6186020	test: 4893.4006439	best: 4893.4006439 (6622)	total: 1m 23s	remaining: 4.77s
6623:	learn: 4200.8521473	test: 4893.1089686	best: 4893.1089686 (6623)	total: 1m 23s	remaining: 4.75s
6624:	learn: 4199.5931196	test: 4892.1074307	best: 4892.1074307 (6624)	total: 1m 23s	remaining: 4.74s
6625:	learn: 4198.9787002	test: 4891.7784444	best: 4891.7784444 (6625)	total: 1m 23s	remaining: 4.73s
6626:	learn: 4197.6516947	test: 4890.6178737	best: 4890.6178737 (6626)	total: 1m 23s	remaining: 4.71s
6627:	learn: 4197.2477699	test: 4890.2778970	best: 4890.2778970 (6627)	total: 1m 23s	remaining: 4.7s
6628:	learn: 4196.0321083	test: 4889.3246977	best: 4889.3246977 (6628)	total: 1m 23s	remaining: 4.69s
6629:	learn: 4194.7552414	test: 4888.4604023	best: 4888.4604023 (6629)	total: 1m 23s	remaining: 4.68s
6630:	learn: 4193.6094719	test: 4888.1191196	best: 4888.1191196 (6630)	total: 1m 23s	remaining: 4.66s
6631:	learn: 4191.7618449	test: 4886.7732461	best: 4886.7732461 (6631)	total: 1m 23

6711:	learn: 4121.0628411	test: 4833.7065595	best: 4833.7065595 (6711)	total: 1m 24s	remaining: 3.64s
6712:	learn: 4120.9964211	test: 4833.3105136	best: 4833.3105136 (6712)	total: 1m 24s	remaining: 3.62s
6713:	learn: 4120.3870886	test: 4832.1719840	best: 4832.1719840 (6713)	total: 1m 24s	remaining: 3.61s
6714:	learn: 4120.2232027	test: 4832.3558193	best: 4832.1719840 (6713)	total: 1m 24s	remaining: 3.6s
6715:	learn: 4119.1541399	test: 4830.5725105	best: 4830.5725105 (6715)	total: 1m 24s	remaining: 3.59s
6716:	learn: 4118.4579401	test: 4830.2030780	best: 4830.2030780 (6716)	total: 1m 24s	remaining: 3.57s
6717:	learn: 4117.6392313	test: 4829.2171772	best: 4829.2171772 (6717)	total: 1m 24s	remaining: 3.56s
6718:	learn: 4117.6120545	test: 4829.2132146	best: 4829.2132146 (6718)	total: 1m 24s	remaining: 3.55s
6719:	learn: 4116.8797834	test: 4829.1956582	best: 4829.1956582 (6719)	total: 1m 24s	remaining: 3.54s
6720:	learn: 4116.2076267	test: 4829.5284831	best: 4829.1956582 (6719)	total: 1m 24

6794:	learn: 4047.0762961	test: 4772.6460079	best: 4772.6460079 (6794)	total: 1m 25s	remaining: 2.59s
6795:	learn: 4046.1765650	test: 4771.5986615	best: 4771.5986615 (6795)	total: 1m 25s	remaining: 2.58s
6796:	learn: 4045.3610579	test: 4770.8073807	best: 4770.8073807 (6796)	total: 1m 25s	remaining: 2.56s
6797:	learn: 4044.9356378	test: 4771.0046043	best: 4770.8073807 (6796)	total: 1m 25s	remaining: 2.55s
6798:	learn: 4044.2312722	test: 4770.9984959	best: 4770.8073807 (6796)	total: 1m 25s	remaining: 2.54s
6799:	learn: 4042.8657247	test: 4768.8272974	best: 4768.8272974 (6799)	total: 1m 25s	remaining: 2.52s
6800:	learn: 4042.2184463	test: 4769.5974096	best: 4768.8272974 (6799)	total: 1m 25s	remaining: 2.51s
6801:	learn: 4041.5664337	test: 4768.8404136	best: 4768.8272974 (6799)	total: 1m 25s	remaining: 2.5s
6802:	learn: 4040.2933005	test: 4768.3537978	best: 4768.3537978 (6802)	total: 1m 25s	remaining: 2.49s
6803:	learn: 4039.8030794	test: 4768.2869805	best: 4768.2869805 (6803)	total: 1m 25

6888:	learn: 3964.6220793	test: 4709.1111343	best: 4709.1111343 (6888)	total: 1m 26s	remaining: 1.4s
6889:	learn: 3963.6300212	test: 4708.2208658	best: 4708.2208658 (6889)	total: 1m 26s	remaining: 1.39s
6890:	learn: 3962.2694456	test: 4707.1692400	best: 4707.1692400 (6890)	total: 1m 27s	remaining: 1.38s
6891:	learn: 3961.9151507	test: 4707.0255355	best: 4707.0255355 (6891)	total: 1m 27s	remaining: 1.36s
6892:	learn: 3960.8824856	test: 4706.5209876	best: 4706.5209876 (6892)	total: 1m 27s	remaining: 1.35s
6893:	learn: 3959.5973042	test: 4704.8102614	best: 4704.8102614 (6893)	total: 1m 27s	remaining: 1.34s
6894:	learn: 3958.3220331	test: 4703.3040948	best: 4703.3040948 (6894)	total: 1m 27s	remaining: 1.32s
6895:	learn: 3957.4581060	test: 4701.8080634	best: 4701.8080634 (6895)	total: 1m 27s	remaining: 1.31s
6896:	learn: 3956.6044716	test: 4701.3190162	best: 4701.3190162 (6896)	total: 1m 27s	remaining: 1.3s
6897:	learn: 3956.0270932	test: 4700.7852486	best: 4700.7852486 (6897)	total: 1m 27s

6977:	learn: 3888.7072713	test: 4659.2166354	best: 4659.2166354 (6977)	total: 1m 28s	remaining: 277ms
6978:	learn: 3888.1508247	test: 4659.2060875	best: 4659.2060875 (6978)	total: 1m 28s	remaining: 265ms
6979:	learn: 3886.8026369	test: 4658.4305820	best: 4658.4305820 (6979)	total: 1m 28s	remaining: 252ms
6980:	learn: 3885.7399217	test: 4658.2790586	best: 4658.2790586 (6980)	total: 1m 28s	remaining: 240ms
6981:	learn: 3884.9758684	test: 4657.5873734	best: 4657.5873734 (6981)	total: 1m 28s	remaining: 227ms
6982:	learn: 3884.3057642	test: 4656.8311629	best: 4656.8311629 (6982)	total: 1m 28s	remaining: 214ms
6983:	learn: 3883.5830001	test: 4656.2204824	best: 4656.2204824 (6983)	total: 1m 28s	remaining: 202ms
6984:	learn: 3883.3013279	test: 4656.1713984	best: 4656.1713984 (6984)	total: 1m 28s	remaining: 189ms
6985:	learn: 3882.3291726	test: 4655.9781889	best: 4655.9781889 (6985)	total: 1m 28s	remaining: 177ms
6986:	learn: 3880.8863636	test: 4654.9568052	best: 4654.9568052 (6986)	total: 1m 2

In [60]:
pd.set_option("display.max_rows", None) 
cat_model.get_feature_importance(data=val_data,prettified=True)


,Feature Id,Importances
0,total_square,7.054933
1,P1_F_max_distance_2_min,5.534761
2,P1_F_square_True_median,5.039829
3,street,3.288416
4,lng,3.131583
5,P1_F_simple_price_median,2.903839
6,F_square_True_max,2.589279
7,reform_mean_floor_count_500,2.462423
8,P1_F_square_False_median,2.258390
9,P1_F_square_True_min,2.030895


In [61]:
price0_preds = cat_model.predict(X1)
metrics_stat(np.array(price0_preds) , np.array(Y1))

{'mape': 0.07083090407179284,
 'mdape': 0.03947519677243707,
 'rmse': 15498.082024727302,
 'r2': 0.9665574836804789,
 'raif_metric': 0.1680572899279168}

In [83]:
price0_preds = cat_model.predict(X_val, ntree_end=7000)
metrics_stat(np.array(Y_val), np.array(price0_preds) /1.15)

{'mape': 0.24420060178052652,
 'mdape': 0.17817126247757128,
 'rmse': 41238.450235036406,
 'r2': 0.7618438512308469,
 'raif_metric': 1.3188323633714687}

In [ ]:
{'mape': 0.24320889449452282,
 'mdape': 0.19656884227825105,
 'rmse': 42808.04249747513,
 'r2': 0.743369739279816,
 'raif_metric': 1.3033049172830355}

In [ ]:
{'mape': 0.3938137450820849,
 'mdape': 0.27994944633239843,
 'rmse': 93738.73635776664,
 'r2': -3.7693751005637317,
 'raif_metric': 2.492385311141158}

{'mape': 0.28628533681632,
 'mdape': 0.21764848060697264,
 'rmse': 84390.73853846012,
 'r2': -1.6484418274393575,
 'raif_metric': 1.552534528099827}

In [ ]:
{'mape': 0.4006764913433361,
 'mdape': 0.28324961990050085,
 'rmse': 94343.39152646794,
 'r2': -4.286807955207906,
 'raif_metric': 2.6756861412533905}

{'mape': 0.2994072204832106,
 'mdape': 0.23136175216191418,
 'rmse': 85205.80509381538,
 'r2': -1.798582387807691,
 'raif_metric': 1.7313695017675064}

In [ ]:
{'mape': 0.45565440900238763,
 'mdape': 0.30249343647803606,
 'rmse': 100782.64525168539,
 'r2': -9.279126502012321,
 'raif_metric': 3.080882843995905}

In [72]:
X1.iloc[:,126:]

,realty_type,street,floor
0,110,S5845,-999
1,10,S27968,-999
2,10,S23597,-999
3,10,S10587,-999
4,10,S11220,-999
5,10,S6752,2.0
6,10,S6703,-999
7,10,S27593,-999
8,10,S19643,-999
9,10,S28820,-999


In [93]:

Xtest1 = Xtest1.fillna(-999)
test_data = Pool(data=Xtest1,
                  cat_features=raw_cat_features)
price0_preds3 = cat_model.predict(test_data)/1.15

In [96]:
price0_preds = (price0_preds1 + price0_preds2 + price0_preds3)/3

In [97]:
pd.DataFrame({"id": [o.row['id'] for o in test_dataset.all_objects],
             "per_square_meter_price": price0_preds
             }).to_csv("ens_for food.csv", index=False)